## Source Code

After [peer reviewing original code in JuliaLang community](https://discourse.julialang.org/t/massive-memory-allocation-on-iterating-algorithm-8-789115-seconds-26-65-m-allocations-7-826-gib-25-07-gc-time/48275/9) I ended up with a much faster (x9.5) code which you can checkout out and use freely [here.](https://github.com/valenmgama/optimization-tec)
***

# TSP Genetic Algorithm Example

For this demonstration we will show how to implement the general idea behind a **Heuristic Genetic Algorithm** to try to find a close to optimal solution for a TSP problem in a short time.

##### Initialize Julia
Import the modules we will be using in our Julia program.

In [1]:
using JuMP, Cbc, Random
using CSV, PrettyTables, Printf

---
### Generate arc cost Matrix
##### Import CSV files
Import `.csv` coordiantes file.
Iterate over each point to calculate the Geometric distance between those two points using _Pythagoras Theorem_.

In [2]:
#Importar datos de Excel
nodes_import = CSV.read("Node_Coordinates.csv", header = true)

#Definir que hay en cada columna del excel y cuantos nodos hay en total
num_nodes = size(nodes_import,1)
idcol = 1
Xcol = 2
Ycol = 3

println(pretty_table(nodes_import))

┌───────┬──────────────┬──────────────┐
│    ID │ X coordinate │ Y coordinate │
│ Int64 │      Float64 │      Float64 │
├───────┼──────────────┼──────────────┤
│     1 │       13.434 │       66.235 │
│     2 │       13.452 │       66.123 │
│     3 │       13.464 │       66.316 │
│     4 │       13.497 │       66.376 │
│     5 │       13.425 │       66.288 │
│     6 │       13.502 │       66.401 │
│     7 │       13.456 │       66.289 │
└───────┴──────────────┴──────────────┘
nothing

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `using DataFrames; CSV.read(input, DataFrame; kw...)
│   caller = ip:0x0
└ @ Core :-1


##### Calculate distance between points
Iterate over each point to calculate the Geometric distance between those two points using _Pythagoras Theorem_.

In [3]:
#Generar la Matriz de distancias a partir del numero de nodos
distance_matrix = Array{Float64}(undef, (num_nodes, num_nodes))
#Llenar la matriz de distancias entre todos los puntos usando Pitagoras
for n in 1:num_nodes
    for s in 1:num_nodes
        distance_matrix[n,s] = sqrt((nodes_import[n,Ycol] - nodes_import[s,Ycol])^2 +
        (nodes_import[n,Xcol] - nodes_import[s,Xcol])^2)
    end
    distance_matrix[n,n] = 0
end

println(pretty_table(round.(distance_matrix, digits=3)))

┌────────┬────────┬────────┬────────┬────────┬────────┬────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │ Col. 4 │ Col. 5 │ Col. 6 │ Col. 7 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│    0.0 │  0.113 │  0.086 │  0.154 │  0.054 │  0.179 │  0.058 │
│  0.113 │    0.0 │  0.193 │  0.257 │  0.167 │  0.282 │  0.166 │
│  0.086 │  0.193 │    0.0 │  0.068 │  0.048 │  0.093 │  0.028 │
│  0.154 │  0.257 │  0.068 │    0.0 │  0.114 │  0.025 │  0.096 │
│  0.054 │  0.167 │  0.048 │  0.114 │    0.0 │  0.137 │  0.031 │
│  0.179 │  0.282 │  0.093 │  0.025 │  0.137 │    0.0 │  0.121 │
│  0.058 │  0.166 │  0.028 │  0.096 │  0.031 │  0.121 │    0.0 │
└────────┴────────┴────────┴────────┴────────┴────────┴────────┘
nothing


This `distance_matrix` will be our **Arc Cost Matrix** when solving the problem.
***

### Build Genetic Definition

##### Solutions as Individuals

In genetic algorithms **a possible solution is presented as an _Individual_** with the possible solution coded in the Individual's _genes_. For this problem the genes will be the sequence or order in which each node will be visited. 

For example in **sequence [2,1,3]** we can say that the 1st node is the second one to be visited, the 2nd node is the starting node, and the 3rd node is visited last. It is implied in the code that follows that there is always an additional arc which returns from the final node to the starting point. In this example from Node 3 to Node 2. The final sequence **would thus be 2-1-3-2.**

In [4]:
struct Individual
    generation::Int
    genes::Vector{Int}
    fitness_score::Float64
end

##### Fitness function

Following the laws of natural selection **we must evaluate and rank each Indivdual's _fitness_.** For optimization problems _fitness_ usually refers directly to the objective function. So for this problem in which we try to _MIN_ the cost of traveling through all nodes the fittest individuals will be the ones whose genes provide the sequence for the path with the lowests costs or _distace_ traveled.

**The fittest individuals will be the ones who will cross with other survivors to produce the next generation of solutions** while the worst solutions, or weakest Individuals, will be eliminated from the population.

In [5]:
function fitness_function(sequence)
    arcs = zeros(Bool, num_nodes, num_nodes)
    i = sequence[1]
    for s in 2:length(sequence)
        j = sequence[s]
        arcs[i,j] = 1
        i = j
    end
    arcs[i,sequence[1]] = 1
    return sum(arcs.*distance_matrix)
end

Individual(gen, genes) = Individual(gen, genes, fitness_function(genes))

Individual

When a new Individual is created it's genes will be automatically evaluated in the `fitness_function`
***

### Run Genetic Algorithm

##### Initialize population

We now define the total size of the population and we **create a starting _population_ with totally random genes**

In [8]:
pop_size = 10

population = Array{Individual}(undef,0)
for i in 1:pop_size
    push!(population, Individual(1,randperm(num_nodes)))
end
sort!(population, by = p -> p.fitness_score)
pop1 = collect(population)

10-element Array{Individual,1}:
 Individual(1, [1, 7, 4, 6, 3, 2, 5], 0.6874149164988359)
 Individual(1, [1, 6, 4, 3, 2, 7, 5], 0.7175549467839349)
 Individual(1, [6, 1, 7, 5, 2, 3, 4], 0.7232520572491363)
 Individual(1, [1, 6, 4, 7, 2, 3, 5], 0.7622498845273068)
 Individual(1, [6, 4, 2, 3, 1, 5, 7], 0.7680690224696226)
 Individual(1, [6, 4, 1, 3, 5, 2, 7], 0.7686382157486541)
 Individual(1, [3, 2, 6, 4, 1, 7, 5], 0.7930989228722776)
 Individual(1, [4, 7, 1, 6, 5, 2, 3], 0.8996583855429939)
 Individual(1, [5, 2, 7, 3, 6, 1, 4], 0.9020340670824505)
 Individual(1, [1, 4, 3, 7, 5, 2, 6], 0.9111300681099338)

##### Chose parents for next generation
Now that this starting population has been _sorted_ from fittests to least fit we must define which and how many of them will be the parents of the next generation.

* **_survivors_** are Individuals who will survive and **mate and pass their genes** on to the next generation.

* **_elites_**, who will also get to **pass directly on to the next generation** elites are also implicitly survivors and will mate with other elite or non-elite survivors.

* **_mutts_**, random individuals in each generation who **suffer a random mutation** or gene change.

As will be shown in a moment, each Survivor will mate twice and father two new Individuals, who will join with the elites to form the next generation. Considering this we must ensure the following equation reamins true to maintain the same population level each generation.

`pop_size = elites + 2*survivors`

In [9]:
pop_size = 10
elites = 2
survivors = 4
mutts = 1

1

#### Iterate Genetic Algorithm for n generations

Follwing this point is main algorith for iterating over generations. For each generation it will:

1. **Sort** the population leaving the _elites_ on top and adding _survivors to the `mating_pool`.

2. **Cross** survivors in random pairs with varying degrees of crossing or gene changes. `cross_point` refers to how many nodes in the gene sequence will be exchanged. Each survivor will mate at least once and will produce 2 _childs_. _childs_ will join the next generation along with the elites.

3. **Mutate** `mutts = 1` random children. The mutation will change one random step in the gene sequence. Ex: 1-3-2-4-1 -> 1-3-4-2-1

4. **Revisar** si la población ya convergio a tener los mismos genes para parar el algortimo o esperar a que se llegue al número de generación establecido.


In [11]:
generations = 6

for gens = 2:generations
    #1st
    ###############################################
    sort!(population, by = p -> p.fitness_score)
    mating_pool = population[1:survivors]
    
    #2nd
    ################################################
    for p in 1:survivors
        parent1_genes = mating_pool[p].genes
        parent2_genes = mating_pool[rand(1:survivors)].genes
        cross_point = rand(1:num_nodes-2)

        child1_genes = parent1_genes[1:cross_point]
        child2_genes = parent2_genes[1:cross_point]
        parent1_left = parent1_genes[cross_point+1:end]
        parent2_left = parent2_genes[cross_point+1:end]

        ch1_pending = Array{Tuple{Int64,Int64}}(undef,0)
        foreach(l -> push!(ch1_pending,
                    (l, findfirst(isequal(l), parent2_genes))),
                    parent1_left)
        sort!(ch1_pending, by = e -> e[:][2])
        foreach(p -> push!(child1_genes, p[1]), ch1_pending)

        ch2_pending = Array{Tuple{Int64,Int64}}(undef,0)
        foreach(l -> push!(ch2_pending,
                    (l, findfirst(isequal(l), parent1_genes))),
                    parent2_left)
        sort!(ch2_pending, by = e -> e[:][2])
        foreach(p -> push!(child2_genes, p[1]), ch2_pending)

        population[elites+p] = Individual(gens,child1_genes)
        population[elites+survivors+p] = Individual(gens,child2_genes)
    end

    #3rd
    ###############################################
    for m in 1:mutts
        mutt = population[rand(elites:num_nodes)]
        mutt_gene = rand(1:num_nodes-1)
        i = findfirst(isequal(mutt_gene), mutt.genes)
        j = findfirst(isequal(mutt_gene+1), mutt.genes)
        mutt.genes[i] = mutt_gene+1
        mutt.genes[j] = mutt_gene
    end
    
    #4th
    ###############################################
    if population[1].fitness_score == population[survivors].fitness_score
        print("Convergencia en la GEN")
        println(gens)
        break
    end
end

Convergencia en la GEN4


In [12]:
pop2 = population

10-element Array{Individual,1}:
 Individual(2, [1, 6, 4, 3, 7, 5, 2], 0.6131673041391617)
 Individual(3, [1, 6, 4, 2, 7, 5, 3], 0.6131673041391617)
 Individual(4, [1, 6, 4, 3, 7, 5, 2], 0.6131673041391617)
 Individual(4, [1, 6, 4, 3, 7, 5, 2], 0.6131673041391617)
 Individual(4, [1, 6, 4, 3, 7, 5, 2], 0.6131673041391617)
 Individual(4, [1, 7, 4, 5, 3, 6, 2], 0.8052035068376027)
 Individual(4, [1, 6, 4, 3, 7, 5, 2], 0.6131673041391617)
 Individual(4, [1, 6, 4, 3, 7, 5, 2], 0.6131673041391617)
 Individual(4, [1, 6, 4, 3, 7, 5, 2], 0.6131673041391617)
 Individual(4, [1, 6, 4, 3, 7, 5, 2], 0.6131673041391617)

In [18]:
final_sequence = []
for n in 1:num_nodes
    push!(final_sequence, findfirst(isequal(n), population[1].genes))
end
push!(final_sequence, findfirst(isequal(1), population[1].genes))

8-element Array{Any,1}:
 1
 7
 4
 3
 6
 2
 5
 1

***
### Analyzing obtained results

From the list of _Individuals_ just presented which corresponds to the last generation in the algorithm we can see that all genes have converged into the sequence, or **solution,  1-7-4-3-6-2-5-1.** With a final _fitness_function_ or **`objetive_function = .613167`**. The only odd _Individual_ is the one who was _mutated_ in this generation.

Let's compare this result to the one obtained in a classic MTZ formulation of the exact same problem.

In [19]:
mtzModel = Model(Cbc.Optimizer)
@variable(mtzModel, rutas[1:num_nodes,1:num_nodes], binary=true)
@variable(mtzModel, 2 <= u[2:num_nodes] <= num_nodes)
@objective(mtzModel, Min, sum(rutas[:,:].*distance_matrix[:,:]))
for i = 1:num_nodes
    @constraint(mtzModel, sum(rutas[i,:]) == 1)
    @constraint(mtzModel, sum(rutas[:,i]) == 1)
    @constraint(mtzModel, rutas[i,i] == 0)
end
for i = 2:num_nodes
    for j = 2:num_nodes
        @constraint(mtzModel, u[i]-u[j]+1 <= (num_nodes-1)*(1-rutas[i,j]))
    end
end
stats = JuMP.optimize!(mtzModel)
print(pretty_table(JuMP.value.(rutas)))

┌────────┬────────┬────────┬────────┬────────┬────────┬────────┐
│ Col. 1 │ Col. 2 │ Col. 3 │ Col. 4 │ Col. 5 │ Col. 6 │ Col. 7 │
├────────┼────────┼────────┼────────┼────────┼────────┼────────┤
│    0.0 │    0.0 │    0.0 │    0.0 │    1.0 │    0.0 │    0.0 │
│    1.0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │
│    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │    1.0 │
│    0.0 │    0.0 │    1.0 │    0.0 │    0.0 │    0.0 │    0.0 │
│    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │    1.0 │    0.0 │
│    0.0 │    0.0 │    0.0 │    1.0 │    0.0 │    0.0 │    0.0 │
│    0.0 │    1.0 │    0.0 │    0.0 │    0.0 │    0.0 │    0.0 │
└────────┴────────┴────────┴────────┴────────┴────────┴────────┘
nothingWelcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 0.415363 - 0.01 seconds
Cgl0002I 7 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 30 strengthened row

***
For this specific problem the obtained solution was not the Optimal but it is pretty closed considering that in the first generation  `best_value = .687` and `worst_value = .911`. 

**Up to about 20 nodes the MTZ formulation may work well buy beyond that point using a GA becomes orders of magnitude faster.**